# 导入库

In [16]:
from autogluon.tabular import TabularDataset, TabularPredictor
import numpy as np
import pandas as pd
from sklearn.utils import shuffle as reset
def train_test_split(data, test_size=0.3, shuffle=True, random_state=None):
    '''Split DataFrame into random train and test subsets
    
    Parameters
    ----------
    data : pandas dataframe, need to split dataset.
    
    test_size : float
        If float, should be between 0.0 and 1.0 and represent the
        proportion of the dataset to include in the train split.
        
    random_state : int, RandomState instance or None, optional (default=None)
        If int, random_state is the seed used by the random number generator;
        If RandomState instance, random_state is the random number generator;
        If None, the random number generator is the RandomState instance used
        by `np.random`.
        
    shuffle : boolean, optional (default=None)
        Whether or not to shuffle the data before splitting. If shuffle=False
        then stratify must be None.
    '''

    if shuffle:
        data = reset(data, random_state=random_state)
	
    train = data[int(len(data)*test_size):].reset_index(drop = True)
    test  = data[:int(len(data)*test_size)].reset_index(drop = True)
    
    return train, test


# 导入数据集

In [18]:
data = TabularDataset('数据集/card_transdata.csv')
#subsample_size = 500  # subsample subset of data for faster demo, try setting this to much larger values
train_data,test_data = train_test_split(data)#= data.iloc[:subsample_size,:]#.sample(n=subsample_size, random_state=0)
train_data.head()

Loaded data from: 数据集/card_transdata.csv | Columns = 8 / 8 | Rows = 1000000 -> 1000000


,distance_from_home,distance_from_last_transaction,ratio_to_median_purchase_price,repeat_retailer,used_chip,used_pin_number,online_order,fraud
0,55.267169,7.482298,1.582675,1.0,0.0,0.0,1.0,0.0
1,58.595702,0.804737,1.189159,1.0,1.0,0.0,1.0,0.0
2,0.969244,0.902882,0.267839,0.0,1.0,0.0,1.0,0.0
3,6.604253,0.629114,2.719961,1.0,1.0,0.0,1.0,0.0
4,20.886200,0.934544,1.037644,1.0,1.0,0.0,0.0,0.0


# 设置预测的行/列

In [21]:
label = 'fraud'
print("Summary of class variable: \n", train_data[label].describe())

Summary of class variable: 
 count    700000.000000
mean          0.087586
std           0.282692
min           0.000000
25%           0.000000
50%           0.000000
75%           0.000000
max           1.000000
Name: fraud, dtype: float64


In [22]:
save_path = 'agModels-predictClass'  # specifies folder to store trained models
hyperparameters = {
    'GBM': [
        {'ag_args_fit': {'num_gpus': 0}},  # Train with CPU
        {'ag_args_fit': {'num_gpus': 1}}   # Train with GPU
    ]
}
predictor = TabularPredictor(label=label).fit(
    train_data,
    hyperparameters=hyperparameters,
    
)
#predictor = TabularPredictor(label=label, path=save_path).fit(train_data)

	Consider setting `time_limit` to ensure training finishes within an expected duration or experiment with a small portion of `train_data` to identify an ideal `presets` and `hyperparameters` configuration.
Beginning AutoGluon training ...
AutoGluon will save models to "agModels-predictClass\"
AutoGluon Version:  0.4.0
Python Version:     3.9.7
Operating System:   Windows
Train Data Rows:    700000
Train Data Columns: 7
Label Column: fraud
Preprocessing data ...
AutoGluon infers your prediction problem is: 'binary' (because only two unique label-values observed).
	2 unique label values:  [0.0, 1.0]
	If 'binary' is not the correct problem_type, please manually specify the problem_type parameter during predictor init (You may specify problem_type as one of: ['binary', 'multiclass', 'regression'])
Selected class <--> label mapping:  class 1 = 1, class 0 = 0
Using Feature Generators to preprocess the data ...
Fitting AutoMLPipelineFeatureGenerator...
	Available Memory:                    20

KeyboardInterrupt: 

In [20]:
#test_data = data.iloc[subsample_size:800,:]#.sample(n=subsample_size, random_state=0)
print(test_data)
y_test = test_data[label]  # values to predict
test_data_nolab = test_data.drop(columns=[label])  # delete label column to prove we're not cheating
test_data_nolab.head()

        distance_from_home  distance_from_last_transaction  \
0                 3.660608                        0.109027   
1                 0.793127                        1.554017   
2                 1.446629                        1.060293   
3                 1.323360                        0.468005   
4                 9.729656                       12.446111   
...                    ...                             ...   
299995            8.158882                        2.793012   
299996           13.564391                        1.487277   
299997            5.417549                        0.374996   
299998            3.342353                        1.044993   
299999           19.202869                       10.797110   

        ratio_to_median_purchase_price  repeat_retailer  used_chip  \
0                             4.304168              1.0        0.0   
1                             0.734684              0.0        0.0   
2                             0.621182       

,distance_from_home,distance_from_last_transaction,ratio_to_median_purchase_price,repeat_retailer,used_chip,used_pin_number,online_order
0,3.660608,0.109027,4.304168,1.0,0.0,0.0,1.0
1,0.793127,1.554017,0.734684,0.0,0.0,0.0,1.0
2,1.446629,1.060293,0.621182,0.0,0.0,0.0,1.0
3,1.323360,0.468005,0.425015,0.0,1.0,0.0,0.0
4,9.729656,12.446111,4.245820,1.0,1.0,0.0,1.0


In [14]:
predictor = TabularPredictor.load(save_path)  # unnecessary, just demonstrates how to load previously-trained predictor from file

y_pred = predictor.predict(test_data_nolab)
print("Predictions:  \n", y_pred)
perf = predictor.evaluate_predictions(y_true=y_test, y_pred=y_pred, auxiliary_metrics=True)

Evaluation: accuracy on test data: 0.9933333333333333
Evaluations on test data:
{
    "accuracy": 0.9933333333333333,
    "balanced_accuracy": 0.9615384615384616,
    "mcc": 0.9572815468117103,
    "f1": 0.9600000000000001,
    "precision": 1.0,
    "recall": 0.9230769230769231
}


Predictions:  
 500    0.0
501    0.0
502    0.0
503    1.0
504    0.0
      ... 
795    0.0
796    0.0
797    0.0
798    0.0
799    0.0
Name: fraud, Length: 300, dtype: float64


In [15]:
predictor.leaderboard(test_data, silent=True)

,model,score_test,score_val,pred_time_test,pred_time_val,fit_time,pred_time_test_marginal,pred_time_val_marginal,fit_time_marginal,stack_level,can_infer,fit_order
0,LightGBM,0.993333,1.00,0.003988,0.004984,0.304152,0.003988,0.004984,0.304152,1,True,4
1,WeightedEnsemble_L2,0.993333,1.00,0.005984,0.004984,0.576394,0.001996,0.000000,0.272241,2,True,14
2,ExtraTreesGini,0.993333,0.96,0.052410,0.039888,0.321642,0.052410,0.039888,0.321642,1,True,8
3,RandomForestGini,0.993333,0.97,0.073796,0.041136,0.506712,0.073796,0.041136,0.506712,1,True,5
4,LightGBMLarge,0.990000,0.99,0.014959,0.004987,0.448746,0.014959,0.004987,0.448746,1,True,13
5,ExtraTreesEntr,0.990000,0.96,0.057999,0.039890,0.417498,0.057999,0.039890,0.417498,1,True,9
6,RandomForestEntr,0.990000,0.97,0.093741,0.035901,0.331518,0.093741,0.035901,0.331518,1,True,6
7,NeuralNetFastAI,0.983333,0.97,0.026925,0.009973,0.475679,0.026925,0.009973,0.475679,1,True,10
8,NeuralNetTorch,0.980000,0.99,0.017951,0.004987,2.923846,0.017951,0.004987,2.923846,1,True,12
9,LightGBMXT,0.976667,0.99,0.007978,0.003988,0.210413,0.007978,0.003988,0.210413,1,True,3
